In [2]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [59]:
#Load page and establish beautiful soup object
html= urlopen('http://www.footballdb.com/fantasy-football/index.html?pos=WR&yr=2012&wk=1&rules=1')
bsobj= BeautifulSoup(html, 'lxml')

#create qb dataframe with column names
columns = ['player_id', 'name', 'team', 'vs', 'ffpts', 'patt', 'pcmp', 'pyds', 'ptds', 'pint', 'p2', 'ruatt', 'ruyds','rutds', 'ru2', 'rec', 'recyds', 'rectd', 'rec2', 'fum', 'fmbtd']
wr = pd.DataFrame(columns = columns, dtype= float)
wr['name'] = wr['name'].astype(str)
wr['team'] = wr['team'].astype(str)
wr['vs'] = wr['vs'].astype(str)

In [ ]:
#Parse html to obtain list of QB's competeing in relevant week 
wr_list= []
for item in (bsobj.tbody({'a':'href'})):
    if '.' not in item.get_text():
        wr_list.append(item.get_text())
    else:
        if item.get_text() == 'A.J. Green':
            wr_list.append(item.get_text())

wr['name'] = wr_list
wr['player_id'] = wr.index + 1

In [63]:
#Parse beautiful soup object to get only data needed in list format
parsed_stats = []
for item in bsobj.tbody.find_all({'td':'class'}):
    parsed_stats.append(item.get_text().split())

In [66]:
#Populate team and vs columns in qb dataframe by sorting parsed_list data
stats = []
team_list = []
vs_list = []
for idx in range(len(parsed_stats)):
    try:
        stats.append(float(parsed_stats[idx][0]))
    except:
        if len(parsed_stats[idx]) > 1:
            team_list.append(parsed_stats[idx][4])
            vs_list.append(parsed_stats[idx+1][0].replace('@', ''))

wr.team, wr.vs = team_list, vs_list

#populate the rest of data frame with state from relevant week
remaining_columns = columns[4:]
for num, col in enumerate(remaining_columns, start=2):
    print(num, col)
    # Flatten the list and assign it to the relevant column
    flat_list = [item for sublist in parsed_stats[num:][::19] for item in sublist]
    wr[col] = flat_list

2 ffpts
3 patt
4 pcmp
5 pyds
6 ptds
7 pint
8 p2
9 ruatt
10 ruyds
11 rutds
12 ru2
13 rec
14 recyds
15 rectd
16 rec2
17 fum
18 fmbtd


In [67]:
wr

,player_id,name,team,vs,ffpts,patt,pcmp,pyds,ptds,pint,...,ruatt,ruyds,rutds,ru2,rec,recyds,rectd,rec2,fum,fmbtd
0,1,Kevin Ogletree,Dal,NYG,23.00,0,0,0,0,0,...,0,0,0,0,8,114,2,0,0,0
1,2,Julio Jones,Atl,KC,22.00,0,0,0,0,0,...,0,0,0,0,6,108,2,0,0,0
2,3,Stephen Hill,NYJ,Buf,20.00,0,0,0,0,0,...,0,0,0,0,5,89,2,0,0,0
3,4,Lance Moore,NO,Was,18.00,0,0,0,0,0,...,0,0,0,0,6,120,1,0,0,0
4,5,Demaryius Thomas,Den,Pit,17.00,0,0,0,0,0,...,0,0,0,0,5,110,1,0,0,0
5,6,Brandon Marshall,Chi,Ind,17.00,0,0,0,0,0,...,0,0,0,0,9,119,1,0,0,0
6,7,Andre Johnson,Hou,Mia,17.00,0,0,0,0,0,...,0,0,0,0,8,119,1,0,0,0
7,8,Pierre Garcon,Was,NO,16.00,0,0,0,0,0,...,0,0,0,0,4,109,1,0,0,0
8,9,Jeremy Maclin,Phi,Cle,15.00,0,0,0,0,0,...,0,0,0,0,7,96,1,0,0,0
9,10,James Jones,GB,SF,14.00,0,0,0,0,0,...,0,0,0,0,4,81,1,0,0,0


In [67]:
for val in stats:
    print(val)

37.0
31.0
23.0
299.0
3.0
0.0
0.0
3.0
25.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
29.0
29.0
22.0
307.0
3.0
1.0
0.0
5.0
12.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
28.0
26.0
19.0
320.0
2.0
0.0
0.0
9.0
42.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
27.0
52.0
24.0
339.0
3.0
2.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
26.0
27.0
19.0
266.0
3.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
26.0
44.0
30.0
303.0
2.0
1.0
1.0
5.0
27.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
24.0
26.0
19.0
253.0
2.0
0.0
1.0
4.0
3.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
23.0
39.0
23.0
260.0
2.0
0.0
1.0
5.0
6.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
23.0
29.0
21.0
299.0
2.0
0.0
0.0
2.0
-1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
23.0
35.0
21.0
333.0
2.0
1.0
0.0
4.0
-4.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
21.0
26.0
20.0
211.0
2.0
0.0
0.0
5.0
13.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
21.0
31.0
23.0
236.0
2.0
0.0
0.0
2.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
19.0
56.0
29.0
317.0
2.0
4.0
0.0
7.0
32.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
19.0
40.0
22.0
245.0
2.0
1.0
0.0
2.0
4.0
0.

In [15]:
bsobj

<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="initial-scale=1.0, width=device-width" name="viewport"/>
<title>2012 Fantasy Football Statistics | The Football Database</title>
<meta content="View week 1 QB fantasy football stats and statistics for the 2012 NFL season. Included are fantasy statistics for passing, rushing, receiving, kicking and defense." name="description"/>
<meta content="index,follow" name="robots"/>
<meta content="FootballDB.com" property="og:site_name"/>
<meta content="2012 Fantasy Football Statistics | The Football Database" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="View week 1 QB fantasy football stats and statistics for the 2012 NFL season. Included are fantasy statistics for passing, rushing, receiving, kicking and defense." property="og:description"/>
<meta content="http://www.footballdb.com/images/footballdb_200x200.png" property="og:i

In [12]:
qb_list

['Matt Ryan',
 'Tony Romo',
 'Robert Griffin III',
 'Drew Brees',
 'Mark Sanchez',
 'Aaron Rodgers',
 'Peyton Manning',
 'Blaine Gabbert',
 'Joe Flacco',
 'Jay Cutler',
 'Alex Smith',
 'Tom Brady',
 'Michael Vick',
 'Ben Roethlisberger',
 'Carson Palmer',
 'Ryan Fitzpatrick',
 'Matt Cassel',
 'Matt Schaub',
 'Philip Rivers',
 'Matthew Stafford',
 'Cam Newton',
 'Eli Manning',
 'Jake Locker',
 'Sam Bradford',
 'Josh Freeman',
 'Russell Wilson',
 'Andrew Luck',
 'Christian Ponder',
 'Kevin Kolb',
 'Andy Dalton',
 'John Skelton',
 'Ryan Tannehill',
 'Tim Tebow',
 'Tyrod Taylor',
 'Colin Kaepernick',
 'Matt Hasselbeck',
 'Bruce Gradkowski',
 'Brandon Weeden']

In [44]:
qb.dtypes

player_id     int64
name         object
team         object
vs           object
ffpts        object
patt         object
pcmp         object
pyds         object
ptds         object
pint         object
p2           object
ratt         object
ryds         object
rtds         object
r2           object
fum          object
td           object
dtype: object